# Tutorial 01: Running Sumo Simulations

This tutorial walks a reader through setting up a simulation environment, initializing an attacking model, and subsequently running a simulation to understand the behavior of the simulation.

## Initialize the simulation environment: The Ring Road


We will beging with one of the simplest traffic simulation environments, the ringroad. On a ring, a finite number of vehicles following one another. First, we must import the proper network file, in this case `RingNetwork`:

In [1]:
from flow.networks.ring import RingNetwork

This network, as well as all other networks in Flow/Anti-Flow, is parametrized by the following arguments: 
* name
* vehicles
* net_params
* initial_config
* traffic_lights

The following code shows how to initialize these quantities:

In [5]:
#Import different needed quantities:
from flow.core.params import VehicleParams
from flow.controllers.car_following_models import IDMController #Human driving model
from flow.controllers.routing_controllers import ContinuousRouter #Router that keeps vehicles on the ring-road
from flow.networks.ring import ADDITIONAL_NET_PARAMS
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import TrafficLightParams
from flow.envs.ring.accel import AccelEnv
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS
from flow.core.params import SumoParams
from flow.core.params import EnvParams
from flow.core.experiment import Experiment

print('Imports succesful.')

Imports succesful.
273.0


We now use the above imported objects to set up out simulation environemnt:

In [12]:
#Simulation parameters:
time_step = 0.1 #In seconds, how far each step of the simulation goes.
ring_length = 300 #The diameter of the ring-road
emission_path = 'data' #Where
want_render = True #If we want SUMO to render the environment and display the simulation.
sim_horizon = 3000 #How many simulation steps will be taken -> Runs for 300 seconds

#initialize the simulation using above parameters:
traffic_lights = TrafficLightParams() #This is empty, so no traffic lights are used.
initial_config = InitialConfig(spacing="uniform", perturbation=1) #Vehicles start out evenly spaced.
vehicles = VehicleParams() #The vehicles object will store different classes of drivers:
sim_params = SumoParams(sim_step=time_step, render=want_render, emission_path=emission_path) #Sets the simulation time-step and where data will be recorded.
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)
net_params = NetParams(additional_params={'length':ring_length,
                                          'lanes':1,
                                          'speed_limit': 30,
                                          'resolution': 40})

#Specify how human driven vehicles will drive:
num_human_drivers = 30
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=num_human_drivers)


#initialize the simulation:
flow_params = dict(
    exp_tag='ring_no_attack',
    env_name=AccelEnv,
    network=RingNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    tls=traffic_lights,
)

# number of time steps
flow_params['env'].horizon = sim_horizon
exp = Experiment(flow_params)
print('Simulation ready.')


Simulation ready.


Let's now run a simulation which doesn't have any malicious vehicles, and only human drivers:

In [13]:
#Run the simulation and save the data:
_ = exp.run(1, convert_to_csv=True)

Round 0, return: 1040.3924964076814
data/ring_no_attack_20211215-0906521639580812.867135-0_emission.csv data
Average, std returns: 1040.3924964076814, 0.0
Average, std velocities: 3.4754782004447096, 0.0
Average, std outflows: 0.0, 0.0
Total time: 37.268765926361084
steps/second: 141.23887210524086


## Visualize simulation results:

First, we need to find the path to the experiment that was just run, which is stored in the exp object. Plotting tools for visualizing the traffic can then be used, which are found in flow\visualize

In [20]:
import os
from flow.visualize.visualize_ring import make_ring_spacetime_fig

emission_location = os.path.join(exp.env.sim_params.emission_path, exp.env.network.name)
print(emission_location)



data/ring_no_attack_20211215-0906521639580812.867135


## Introduce an attacker:

Here we use the compromised model for an ACC which executes a randomized deceleration event (RDA). A single ACC vehicle is defined which has a given attack mangitude and duration. The attacker waits for either 50 seconds, or to travel 900 m before it executes its attack, and the attack is only executed once.

In [ ]:
from flow.controllers.adversarial_car_following_model import ACC_Switched_Controller_Attacked_Single

attack_duration = 10
attack_magnitude = -1.5
adversary_ACC_controller = (ACC_Switched_Controller_Attacked_Single,{
    'warmup_steps':50,
    'distance_threshold_max':910,
    'Total_Attack_Duration':attack_duration,
    'attack_decel_rate':attack_magnitude,
    'display_attack_info':display_attack_info,
    'V_m':15.0})

# Reinstantiate which vehicles are  being run:
vehicles = VehicleParams()


In [ ]:
import pandas as pd

pd.read_csv(emission_location + '-emission.csv')

As you can see, each row contains vehicle information for a certain vehicle (specified under the *id* column) at a certain time (specified under the *time* column). These information can then be used to plot various representations of the simulation, examples of which can be found in the `flow/visualize` folder.

## 6. Modifying the Simulation
This tutorial has walked you through running a single lane ring road experiment in Flow. As we have mentioned before, these simulations are highly parametrizable. This allows us to try different representations of the task. For example, what happens if no initial perturbations are introduced to the system of homogenous human-driven vehicles?

```
initial_config = InitialConfig()
```

In addition, how does the task change in the presence of multiple lanes where vehicles can overtake one another?

```
net_params = NetParams(
    additional_params={
        'length': 230, 
        'lanes': 2, 
        'speed_limit': 30, 
        'resolution': 40
    }
)
```

Feel free to experiment with all these problems and more!

## Bibliography
[1] Sugiyama, Yuki, et al. "Traffic jams without bottlenecks—experimental evidence for the physical mechanism of the formation of a jam." New journal of physics 10.3 (2008): 033001.

[2] Treiber, Martin, Ansgar Hennecke, and Dirk Helbing. "Congested traffic states in empirical observations and microscopic simulations." Physical review E 62.2 (2000): 1805.